# Automatic estimate the patch size

**Patch size** is an important parameter to extract local image motifs, compute rotatioanl and reflectional symmetry maps. 

In [ ]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

#from mtflearn import ZPs

In [2]:
from mtflearn.features import autocorrelation, radial_profile, get_characteristic_length, get_characteristic_length_fft
from mtflearn.utils import normalize_image

In [3]:
from skimage.transform import resize

## Load image

In [4]:
filename = 'data\\monolayer_MoSe2_80K.npy'
img = np.load(filename)[0:800, 0:800]
img = resize(img, (512, 512))
img = normalize_image(img, 0, 1)

## Method 1: Estimating the Lattice Constant from Fourier Space

Given an image $I$, we follow these steps to estimate the characteristic length from the Fourier space:

1. **Compute the Power Spectrum**:
   - Calculate the power spectrum of the image $I$, denoted as $F$.

2. **Radial Averaging**:
   - Compute the radial average of $F$, denoted as $L$.

3. **Baseline Correction**:
   - Perform baseline correction on $L$, resulting in a corrected curve denoted as $L'$.

4. **Peak Detection**:
   - Identify the dominant peak in $L'$. The position of this peak is denoted as $k$.

5. **Conversion to Real Space**:
   - Convert the peak position $k$ to a real-space length $s$.

This method allows us to estimate the lattice constant by analyzing the frequency domain representation of the image and extracting the most prominent spatial frequency, which is then translated into a characteristic length in real space.

In [7]:
s = get_characteristic_length_fft(img)
a1 = 2/np.sqrt(3) * s
print('Lattice constant is {} pixels.'.format(a1))

## Method 2: Estimating the Lattice Constant from the Radial Distribution Function

Given an image $I$, we follow these steps to estimate the characteristic length from the radial distribution function:

1. **Compute the Autocorrelation**:
   - Calculate the autocorrelation of the image $I$, denoted as $A$.

2. **Radial Averaging**:
   - Compute the radial average of $A$, denoted as $L$.

3. **Peak Detection**:
   - Locate the first peak in $L$. The position of this peak is denoted as $s$.

This method enables us to estimate the lattice constant by analyzing the spatial correlations within the image, identifying the most prominent distance over which the image exhibits periodicity.

In [8]:
a2 = get_characteristic_length(img)

print('Lattice constant is {} pixels.'.format(a2))